In [1]:
# 网络中的网络
# http://zh.gluon.ai/chapter_convolutional-neural-networks/nin-gluon.html

from mxnet.gluon import nn

def mlpconv(channels, kernel_size, padding,
           strides=1, max_pooling=True):
    out = nn.Sequential()
    out.add(
        nn.Conv2D(channels=channels, kernel_size=kernel_size,
                 strides=strides, padding=padding,
                 activation='relu'),
        nn.Conv2D(channels=channels, kernel_size=1,
                 strides=1, padding=0, activation='relu'),
        nn.Conv2D(channels=channels, kernel_size=1,
                 strides=1, padding=0, activation='relu')
    )
    if max_pooling:
        out.add(nn.MaxPool2D(pool_size=3, strides=2))
    return out

/home/ly/anaconda3/envs/learning/lib/python3.6/site-packages/h5py/__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters


In [2]:
from mxnet import nd

blk = mlpconv(64, 3, 1)
blk.initialize()

x=nd.random.uniform(shape=(32, 3, 16, 16))
y = blk(x)
print(y.shape)
print(blk)

(32, 64, 7, 7)
Sequential(
  (0): Conv2D(3 -> 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  (1): Conv2D(64 -> 64, kernel_size=(1, 1), stride=(1, 1))
  (2): Conv2D(64 -> 64, kernel_size=(1, 1), stride=(1, 1))
  (3): MaxPool2D(size=(3, 3), stride=(2, 2), padding=(0, 0), ceil_mode=False)
)


In [3]:
net = nn.Sequential()
# add name_scope on the outer most Sequential
with net.name_scope():
    net.add(
        mlpconv(96, 11, 0, strides=4),
        mlpconv(256, 5, 2),
        mlpconv(384, 3, 1),
        nn.Dropout(.5),
        # 目标类为10类
        mlpconv(10, 3, 1, max_pooling=False),
        # 输入为 batch_size x 10 x 5 x 5, 通过AvgPool2D转成
        # batch_size x 10 x 1 x 1。
        # 我们可以使用 nn.AvgPool2D(pool_size=5),
        # 但更方便是使用全局池化，可以避免估算pool_size大小
        nn.GlobalAvgPool2D(),
        # 转成 batch_size x 10
        nn.Flatten()
    )
print(net)

Sequential(
  (0): Sequential(
    (0): Conv2D(None -> 96, kernel_size=(11, 11), stride=(4, 4))
    (1): Conv2D(None -> 96, kernel_size=(1, 1), stride=(1, 1))
    (2): Conv2D(None -> 96, kernel_size=(1, 1), stride=(1, 1))
    (3): MaxPool2D(size=(3, 3), stride=(2, 2), padding=(0, 0), ceil_mode=False)
  )
  (1): Sequential(
    (0): Conv2D(None -> 256, kernel_size=(5, 5), stride=(1, 1), padding=(2, 2))
    (1): Conv2D(None -> 256, kernel_size=(1, 1), stride=(1, 1))
    (2): Conv2D(None -> 256, kernel_size=(1, 1), stride=(1, 1))
    (3): MaxPool2D(size=(3, 3), stride=(2, 2), padding=(0, 0), ceil_mode=False)
  )
  (2): Sequential(
    (0): Conv2D(None -> 384, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (1): Conv2D(None -> 384, kernel_size=(1, 1), stride=(1, 1))
    (2): Conv2D(None -> 384, kernel_size=(1, 1), stride=(1, 1))
    (3): MaxPool2D(size=(3, 3), stride=(2, 2), padding=(0, 0), ceil_mode=False)
  )
  (3): Dropout(p = 0.5, axes=())
  (4): Sequential(
    (0): Conv2D(None

In [4]:
# 读取数据
import sys
sys.path.append('..')
import utils
from mxnet import autograd
from mxnet import gluon
from mxnet import nd
from mxnet import image
from mxnet import init

batch_size = 64
resize=224

ctx = utils.try_gpu(1)
net.initialize(ctx=ctx, init=init.Xavier())

x = nd.random.uniform(shape=(batch_size, 1, resize, resize), ctx=ctx)
y = net(x)
print(y.context)
print(net)

def transform(data, label):
#     print(data.shape)   # (28, 28, 1)
#     print(label.shape)    # (1,)
# change data from batch x height x width x channel
# to batch x channel x height x width
#     return nd.transpose(data.astype('float32'), (0,3,1,2))/255, label.astype('float32')
    data = image.imresize(data, resize, resize)
#     data = new_data
            
    return nd.transpose(data.astype('float32'), (2,0,1))/255, label.astype('float32')
#     return data.astype('float32') / 255, label.astype('float32')
mnist_train = gluon.data.vision.FashionMNIST(train=True, transform=transform)
mnist_test = gluon.data.vision.FashionMNIST(train=False, transform=transform)
train_data = gluon.data.DataLoader(mnist_train, batch_size, shuffle=True)
test_data = gluon.data.DataLoader(mnist_test, batch_size, shuffle=False)

gpu(1)
Sequential(
  (0): Sequential(
    (0): Conv2D(1 -> 96, kernel_size=(11, 11), stride=(4, 4))
    (1): Conv2D(96 -> 96, kernel_size=(1, 1), stride=(1, 1))
    (2): Conv2D(96 -> 96, kernel_size=(1, 1), stride=(1, 1))
    (3): MaxPool2D(size=(3, 3), stride=(2, 2), padding=(0, 0), ceil_mode=False)
  )
  (1): Sequential(
    (0): Conv2D(96 -> 256, kernel_size=(5, 5), stride=(1, 1), padding=(2, 2))
    (1): Conv2D(256 -> 256, kernel_size=(1, 1), stride=(1, 1))
    (2): Conv2D(256 -> 256, kernel_size=(1, 1), stride=(1, 1))
    (3): MaxPool2D(size=(3, 3), stride=(2, 2), padding=(0, 0), ceil_mode=False)
  )
  (2): Sequential(
    (0): Conv2D(256 -> 384, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (1): Conv2D(384 -> 384, kernel_size=(1, 1), stride=(1, 1))
    (2): Conv2D(384 -> 384, kernel_size=(1, 1), stride=(1, 1))
    (3): MaxPool2D(size=(3, 3), stride=(2, 2), padding=(0, 0), ceil_mode=False)
  )
  (3): Dropout(p = 0.5, axes=())
  (4): Sequential(
    (0): Conv2D(384 -> 10, 

In [5]:
# 训练
import time
from mxnet import gluon

softmax_cross_entropy = gluon.loss.SoftmaxCrossEntropyLoss()

trainer = gluon.Trainer(net.collect_params(), 'sgd', {
    'learning_rate': 0.1
})

print_net = False

for epoch in range(10):
    time_start = time.time()
    train_loss = 0.
    train_acc = 0.
    for data, label in train_data:
        label = label.as_in_context(ctx)
        with autograd.record():
            output = net(data.as_in_context(ctx))
            loss = softmax_cross_entropy(output, label)
        if print_net:
            print(net)
            print_net = False
        loss.backward()
        trainer.step(batch_size)

        train_loss += nd.mean(loss).asscalar()
        train_acc += utils.accuracy(output, label)
    test_acc = utils.evaluate_accuracy(test_data, net, ctx)
    print("Epoch %d. Loss: %.4f, Train acc %.4f, Test acc %.4f, Time %.0f sec" % (
        epoch, train_loss/len(train_data),
        train_acc/len(train_data), test_acc, time.time() - time_start))

Epoch 0. Loss: 1.6594, Train acc 0.3832, Test acc 0.6424, Time 116 sec
Epoch 1. Loss: 0.6830, Train acc 0.7457, Test acc 0.8108, Time 113 sec
Epoch 2. Loss: 0.5065, Train acc 0.8120, Test acc 0.8230, Time 112 sec
Epoch 3. Loss: 0.4361, Train acc 0.8385, Test acc 0.8406, Time 113 sec
Epoch 4. Loss: 0.3911, Train acc 0.8558, Test acc 0.8613, Time 114 sec
Epoch 5. Loss: 0.3613, Train acc 0.8654, Test acc 0.8814, Time 113 sec
Epoch 6. Loss: 0.3414, Train acc 0.8747, Test acc 0.8860, Time 114 sec
Epoch 7. Loss: 0.3201, Train acc 0.8830, Test acc 0.8911, Time 115 sec
Epoch 8. Loss: 0.3041, Train acc 0.8886, Test acc 0.8939, Time 111 sec
Epoch 9. Loss: 0.2898, Train acc 0.8942, Test acc 0.9045, Time 112 sec
